# Selección de características en modelos predictivos

Este proyecto es para IA de Moisés Calzado Cobo y Antonio Germán Márquez Trujillo

Estos archivos contienen los algoritmos implementados, pero no los importamos ya que en esta libreta los implementamos explicando su funcionamiento


In [1]:
import pandas as pandas
import time
import numpy as np
import matplotlib.pyplot as plt
import multiprocessing as mp
from sklearn.model_selection import cross_val_score
from sklearn import tree
from funcy import join
from sklearn import preprocessing
pandas.set_option('max_colwidth', 800)

In [2]:
datos = pandas.read_csv("datos/titanic.csv")
#datos = pandas.read_csv("datos/BreastCancerDataset.csv")
datos.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Initial,Age_band,Family_Size,Alone,Fare_cat,Deck,Title,Is_Married,Survived
0,3,0,22.0,1,0,7.2500,0,0,1,1,0,0,7,11,0,0
1,1,1,38.0,1,0,71.2833,1,1,2,1,0,3,2,12,1,1
2,3,1,26.0,0,0,7.9250,0,2,1,0,1,1,7,8,0,1
3,1,1,35.0,1,0,53.1000,0,1,2,1,0,3,2,12,1,1
4,3,0,35.0,0,0,8.0500,0,0,2,0,1,1,7,11,0,0


In [3]:
def evaluar_soluciones(datos, solucion_actual, nueva_variable, objetivo, n_exp, cv):
    variables = solucion_actual[:]
    variables.append(nueva_variable)
    data_frame = pandas.DataFrame(data=datos)
    X = data_frame[variables]
    y = data_frame[objetivo]
    clf = tree.DecisionTreeClassifier()
    
    scores = cross_val_score(clf, X, y, cv=cv, scoring="balanced_accuracy")
    
    for i in range(n_exp-1):
        new_scores = cross_val_score(clf, X, y, cv=cv, scoring="balanced_accuracy") 
        scores = scores + new_scores
    
    scores = scores/n_exp
    clave = ', '.join(variables)
    
    diccionario_resultado = {}
    diccionario_resultado[clave] = np.mean(scores)
    return diccionario_resultado

In [45]:
def evaluar_soluciones_eliminando(datos, solucion_actual, variable_a_eliminar, objetivo, n_exp, cv):
    variables = solucion_actual[:]
    variables.remove(variable_a_eliminar)
    if len(variables) < 1:
        return
    data_frame = pandas.DataFrame(data=datos)
    X = data_frame[variables]
    y = data_frame[objetivo]
    clf = tree.DecisionTreeClassifier()
    
    scores = cross_val_score(clf, X, y, cv=cv, scoring="balanced_accuracy")
    
    for i in range(n_exp-1):
        new_scores = cross_val_score(clf, X, y, cv=cv, scoring="balanced_accuracy") 
        scores = scores + new_scores
    
    scores = scores/n_exp
    clave = ', '.join(variables)
    
    diccionario_resultado = {}
    diccionario_resultado[clave] = np.mean(scores)
    return diccionario_resultado

Algoritmo SFS paralelizado:

In [4]:
def SFS(datos, respuesta, d = 0):
    start = time.time()

    diccionario_resultado = {}
    columnas = list(datos.columns)
    columnas.remove(respuesta)
    solucion_actual = []

    k = 0
    d = d if d else len(columnas)
    
    while(k<d):
        pool = mp.Pool(mp.cpu_count())
        new_resultados = pool.starmap(evaluar_soluciones, [(datos, solucion_actual, nuevaVariable, respuesta, 15, 10) for nuevaVariable in columnas])
        pool.close()
        resultado = join(new_resultados)
        
        variable_escogida = max(resultado, key=resultado.get)
        variables_nuevas = variable_escogida.split(", ")
        solucion_actual.append(variables_nuevas[len(variables_nuevas)-1])
        columnas.remove(variables_nuevas[len(variables_nuevas)-1])

        k = k+1
        
        diccionario_resultado[variable_escogida] = resultado[variable_escogida]
    
    done = time.time()
    elapsed = done - start
    print("Tiempo empleado: ", elapsed)
    return diccionario_resultado

In [5]:
resultado_SFS = SFS(datos, "Survived", 3)
print(resultado_SFS)

Tiempo empleado:  14.498463153839111
{'Initial': 0.7833541295306, 'Initial, SibSp': 0.8052182893359363, 'Initial, SibSp, Deck': 0.8077201991907874}


Algoritmo SFFS paralelizado:

In [58]:
def SFFS(datos, respuesta):
    start = time.time()
    diccionario_resultado = {}
    soluciones_actual = []
    añadidos = []
    eliminados = []
    columnas = list(datos.columns)
    k = 0
    
    #Compruebo que la variable a predecir no esté en mi conjunto de variables a evaluar
    if respuesta in columnas:
        columnas.remove(respuesta)
    
    while(k<10):
        resultado = []
        score_resultado = 0
        score_resultado_eliminado = 0
        resultado_eliminado = []
        eliminado = ''
        
        #Actualizo el listado de columnas que tengo que evaluar
        columnas_a_evaluar = [x for x in columnas if (x not in añadidos and x not in soluciones_actual)]
        
        #Calculamos el nuevo resultado óptimo
        pool = mp.Pool(mp.cpu_count())
        new_resultados = pool.starmap(evaluar_soluciones, [(datos, soluciones_actual, nuevaVariable, respuesta, 15, 10) for nuevaVariable in columnas_a_evaluar])
        pool.close()
        resultado = join(new_resultados)
        variable_escogida = max(resultado, key=resultado.get)
        variables_nuevas = variable_escogida.split(", ")

        score_resultado = resultado[variable_escogida]
        
        # Añadimos a nuestras soluciones la variable escogida
        soluciones_actual.append(variables_nuevas[len(variables_nuevas)-1])
        añadidos.append(variables_nuevas[len(variables_nuevas)-1])
        score_resultado_eliminado = score_resultado
                
        # Pasamos a eliminar variables para comprobar si tengo mejores resultados
        if len(soluciones_actual) > 1:
            variables_a_eliminar = [x for x in soluciones_actual if x not in eliminados]

            pool = mp.Pool(mp.cpu_count())
            new_resultados = pool.starmap(evaluar_soluciones_eliminando, [(datos, soluciones_actual, variable_a_eliminar, respuesta, 15, 10) for variable_a_eliminar in variables_a_eliminar])
            pool.close()
            resultado = join(new_resultados)
            variable_a_eliminar = max(resultado, key=resultado.get)
            variables_nuevas = variable_a_eliminar.split(", ")
            
            score_resultado_eliminado = resultado[variable_a_eliminar]
            
        if score_resultado<score_resultado_eliminado:
            soluciones_actual.remove(variables_nuevas[len(variables_nuevas)-1])
            eliminados.append(variables_nuevas[len(variables_nuevas)-1])
        
        k = k+1
        
        if len(añadidos)<len(columnas):
            clave = ', '.join(soluciones_actual)
            diccionario_resultado[clave] = score_resultado_eliminado
    done = time.time()
    elapsed = done - start
    print("Tiempo empleado: ", elapsed)
    return diccionario_resultado        

In [59]:
resultado_SFFS = SFFS(datos, "Survived")

Tiempo empleado:  118.58071088790894


In [60]:
def mostrar_datos(datos):
    datos_tabla = [['Mejor Solución', 'Tamaño', 'Rendimiento']]
    claves = datos.keys()
    
    for clave in claves:
        length = len(clave.split(", "))
        datos_tabla = datos_tabla +[[clave, length, datos[clave]]]

    return pandas.DataFrame(datos_tabla)

In [61]:
mostrar_datos(resultado_SFFS)

,0,1,2
0,Mejor Solución,Tamaño,Rendimiento
1,Initial,1,0.783354
2,"Initial, SibSp",2,0.783354
3,"Initial, SibSp, Deck",3,0.805218
4,"Initial, SibSp, Deck, Fare_cat",4,0.80772
5,"Initial, SibSp, Deck, Fare_cat, Title",5,0.809327
6,"Initial, SibSp, Deck, Fare_cat, Title, Sex",6,0.815809
7,"Initial, SibSp, Deck, Fare_cat, Title, Is_Married",6,0.815904
8,"Initial, SibSp, Deck, Fare_cat, Title, Alone",6,0.815714
9,"Initial, SibSp, Deck, Fare_cat, Title, Pclass",6,0.810742
